### Exercise 8.3

Modify the previous marker-based code by introducing non-uniform distances between
nodal points and markers and by using a bisection algorithm (Fig. 8.10) to define
indices of the two nearest nodes. Prescribe a slightly variable velocity on the nodal
points and interpolate it to markers when displacing them. An example is in
Markers_1Dirregular.m.

In [ ]:
import numpy as np
import random
from matplotlib import pyplot as plt
from matplotlib import gridspec

#### Step 1: Build the mesh and prescribe the initial wave form

Use random function to generate non-uniform mesh and variable velocity.
The mesh generated has a resolution of (0.75 - 1.25) * xstp.
And the velocity field has a magnitude of (0.5 - 1.5) * vx

In [ ]:
xsize = 150.0 # m
xnum = 151
xstp = xsize / (xnum-1)
xs = np.linspace(0.0, 150.0, xnum)
for i in range(xnum):  
    # generate non-uniform spacing mesh
    xs[i] += (random.uniform(0.0, 1.0) - 0.5) * xstp / 4.0

vx0 = 1 # 1m/s
ntimes = 800  # count of time steps
dx = 60.0  # total distance
t = dx / vx0
dt = xstp / vx0 * dx / ntimes  # this corresponds to a small coefficient in the Corant condition
vxs = np.zeros(xnum)
for i in range(xnum):
    # generate variable velocity
    vxs[i] = vx0 * (random.uniform(0.0, 1.0) + 0.5)

def get_initial_density(xs):
    '''
    get initial density on x coordinates
    Inputs:
        xs (list of double): x coordinates
    return:
        rho0s (list of doubles): initial densities
    '''
    xnum = xs.size
    rhos0 = np.zeros(xnum)
    # Defining initial density distribution 
    for i in range(xnum):
        # Background density
        rhos0[i] = 3000.0
        # Square wave
        if xs[i]>=3 and xs[i]<=23:
            rhos0[i]=3300
        # Triangular wave
        if xs[i] >= 43 and xs[i] <= 53:
            rhos0[i] = 3000+ (xs[i]-43) / 10 * 300.0
        if xs[i] >=53 and xs[i] <=63:
            rhos0[i] = 3300 - (xs[i]-53) / 10 * 300.0
    return rhos0


rhos0 = get_initial_density(xs)  # get initial density on nodes

### Step 2: Advection method

#### Issue of vary small value encoutered in the weight of averaging

Note that there is issue encountered by the interpolation of mesh property when the number of markers is 200.
Here is the warning message.

    /tmp/ipykernel_2805009/3036365379.py:68: RuntimeWarning: invalid value encountered in double_scalars
        rhos[i] /= sum_dist

Is it due to a near 0.0 value? yes
It will come with the warning message I tried to shoot:

    "marker_in_cell_property: vary small value encountered in the weight of averaging."

When the number of markers is increased to 500, this is not an issue anymore.

In [ ]:
def nodeL_index(x, xs):
    '''
    Get the nearest node to the left of a point x
    '''
    xnum = xs.size
    id0 = 0
    id1 = xnum-1
    while (id1 - id0 > 1):
        id_mid = int((id0 + id1) / 2.0)
        x_mid = xs[id_mid]
        if x_mid < x:
            id0 = id_mid
        else:
            id1 = id_mid
    return id0


def marker_in_cell_advection(xs, xms0, vxs, dt):
    '''
    marker in cell advection scheme
    Inputs:
        xs (list of float): x coordinates
        xms0 (list of float): x coordinates of markers in the previous step
        vxs: velocity
        dt: time step
    Returns:
        xms (list of float): x coordinates of markers
    '''
    xsize = xs[-1] - xs[0]  # size of the domain
    mnum = xms0.size
    xms = np.zeros(mnum)
    ids_on_nodes = [[] for i in range(xnum)]
    for i in range(mnum):
        x_marker = xms0[i]
        n0 = nodeL_index(x_marker, xs) # index of the node to the left of the marker
        xstp = xs[n0+1] - xs[n0]
        vxi = vxs[n0]*(xs[n0+1] - x_marker)/xstp + vxs[n0+1]*(x_marker - xs[n0])/xstp
        xms[i] = xms0[i] + vxi * dt
        if xms[i] < 0.0:
            # recycle left side
            xms[i] += xsize
        elif xms[i] > xsize:
            # recycle right side
            xms[i] -= xsize
        n = nodeL_index(xms[i], xs) # add the index of this marker to nodes to the left and right
        ids_on_nodes[n].append(i)
        ids_on_nodes[n+1].append(i)
    return xms, ids_on_nodes


def marker_in_cell_property(xs, xms, rho_of_markers, ids_on_nodes):
    '''
    weight averaging of densities on markers to get the density field on nodes
    Inputs:
        xs (list of float): x coordinates
        xms (list of float): x coordinates of markers
        rho_of_markers: density of markers
        ids_on_nodes: ids of markers near a node
    '''
    xnum = xs.size
    rhos = np.zeros(xnum)
    for i in range(xnum):
        sum_dist = 0.0
        for id in ids_on_nodes[i]:
            dist = abs(xms[id] - xs[i])
            rhos[i] += rho_of_markers[id] * dist
            sum_dist += dist
        if abs(sum_dist) < 1e-16:
            print("marker_in_cell_property: vary small value encountered in the weight of averaging.")
        rhos[i] /= sum_dist
    return rhos

#### Step 3: advection

Here, an array is initiated for each method.
Advection is handled by looping the time steps.

In [ ]:
rhos_mic = rhos0.copy()
n_marker = 500  # number of markers
xms = np.linspace(0, xsize, n_marker)  # initial positions of markers
rho_of_markers = get_initial_density(xms)  # initial density of markers

# advection by time step
for i in range(ntimes):
    xms, ids_on_nodes = marker_in_cell_advection(xs, xms, vxs, dt)
    rhos_mic = marker_in_cell_property(xs, xms, rho_of_markers, ids_on_nodes)



#### Plot

In [ ]:
fig = plt.figure(tight_layout=True, figsize=(5, 10))
gs = gridspec.GridSpec(2, 1)
ax = fig.add_subplot(gs[0, 0])  # initial
ax.plot(xs, rhos0, 'b')
ax.set_xlabel("x [m]")
ax.set_ylabel("Density [kg/m3]")
ax.set_xlim(0.0, 80.0)
ax.set_title("t = 0.0 s")
ax = fig.add_subplot(gs[1, 0])  # method in cell
ax.plot(xs, rhos_mic, 'r.', label="marker in cell")
ax.set_xlabel("x [m]")
ax.set_ylabel("Density [kg/m3]")
ax.set_xlim(dx, 80.0 + dx)
ax.set_title("MiC, t = %.1f s" % t)
ax.legend()
fig.show()